In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline
def savefig(name):
    plt.savefig(name + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(name + ".pdf", bbox_inches='tight')

In [ ]:
# reload(pu_delayfb)
# carbons = np.arange(1,8)

# single_C_decay_data = dict()
# single_C_decay_data_T2star = dict()

# for c in carbons:
#     data = dict()
#     data["x"], data["y"], data["y_u"], data["fr"] = \
#     pu_delayfb.number_of_repetitions_stitched(
#         multi_contains = ['sweep_number_of_reps_C%d_X' % (c)],
#         do_fit = True,
#         fixed=[0,2,5,6,4], 
#         ret_data_fit=True,
#         T2star_correction=True,
#         LDE_element_length=7e-6
#     )
#     single_C_decay_data[str(c)] = data

In [ ]:
reload(pu_delayfb)
carbons = [1,2,3,4,5,6,7]

single_C_decay_data = dict()

for c in carbons:
    data = dict()
    data["x"], data["y"], data["y_u"], data["fr"] = \
    pu_delayfb.number_of_repetitions_stitched(
        multi_contains = ['sweep_number_of_reps_C%d_X' % (c)],
        older_than="20170812000000",
        do_fit = True,
        fixed=[0,2,5,6,4], 
        ret_data_fit=True,
        T2star_correction=True,
        LDE_element_length=7e-6,
    )
    single_C_decay_data[str(c)] = data

In [ ]:
reload(pu_delayfb)
fb_results = {}
tomo_list = ['X', 'Y']

for c in carbons:
    c_key = str(c)
    fb_results[c_key] = {}

    folder_names = []
    for d_i in range(2):
        name_template = "phase_fb_delayline_C%d_sec%d" % (
            c,
            d_i
        )
        folder_names += [name_template]
    print folder_names
    for t in tomo_list:
        print("Tomo: %s" % t)
        data = dict()
        data["x"], data["y"], data["y_u"], data["fr"] = \
        pu_delayfb.number_of_repetitions_stitched(
            contains="",
            multi_contains=folder_names,
            older_than='20170816000000',
            do_fit=True,
            x_only=True,
            tomo_basis=t,
            fit_n=1.0,
            fixed=[0,2,5,6,4],
            ret_data_fit=True,
            T2star_correction=True,
            LDE_element_length=7e-6
        )
        fb_results[c_key][t] = data

In [ ]:
# reload(pu_delayfb)
# carbons = np.arange(1,8)

# repump_time_data = dict()

# for c in carbons:
#     data = dict()
#     data["x"], data["y"], data["y_u"], data["fr"] = \
#     pu_delayfb.average_repump_time(contains = 'Sweep_Repump_time_C%d_X' % (c),do_fit = True,fixed=[], ret_data_fit=True)
#     repump_time_data[str(c)] = data

In [ ]:
import itertools
reload(pu_delayfb)

carbons = np.arange(1,8)
carbon_combis = list(itertools.combinations(carbons, 2))

print("All carbon combinations: " + str(carbon_combis))
print("Selected carbon combinations: " + str(carbon_combis))

C13_X_phase = 0.0

msmt_sweep_limits = [
    (10, 190, 30),
    (190, 340, 30),
    (340, 850, 120),
]

msmt_templates = [
    {
        'carbon_encoding':              'serial_swap',
        'carbon_swap_el_states':        ['Z', 'Z'],
        'Tomography_list': [
            # ['X', 'I'],
            # ['I', 'X'],
            ['X', 'X'],
            [C13_X_phase + 45.0, C13_X_phase + 45.0],
            [C13_X_phase + 45.0, C13_X_phase - 45.0],
        ]
    },
    {
        'carbon_encoding':              'MBE',
        'carbon_swap_el_states':        ['X'],
        'Tomography_list': [
            ['X', 'X'],
            ['Y', 'Y'],
            ['Z', 'Z'],
            ['X', 'Y'],
            ['Y', 'X'],
        ]
    },
    {
        'carbon_encoding':              'MBE',
        'carbon_swap_el_states':        ['-X'],
        'Tomography_list': [
            ['X', 'X'],
            ['Y', 'Y'],
            ['Z', 'Z'],
            ['X', 'Y'],
            ['Y', 'X'],
        ]
    }
]

results = {}

for combi in carbon_combis:
    combi_key = "".join([str(c) for c in combi])
    results[combi_key] = {}
    for m in msmt_templates:
        dictkey = "%s_%s" % (m['carbon_encoding'], "".join(m['carbon_swap_el_states']))
        results[combi_key][dictkey] = {}
        folder_names = []
        for d_i, d in enumerate(msmt_sweep_limits):
            name_template = "phase_fb_delayline_C%s_%s_%s_sec%d" % (
                    "".join([str(c) for c in combi]),
                    m['carbon_encoding'],
                    "".join(m['carbon_swap_el_states']),
                    d_i
            )
            folder_names += [name_template]
        print folder_names
        for t in ["".join([str(tbasis) for tbasis in tbases]) for tbases in m['Tomography_list']]:
            print("Tomo: %s" % t)
            data = dict()
            data["x"], data["y"], data["y_u"], data["fr"] = \
            pu_delayfb.number_of_repetitions_stitched(
                contains="",
                multi_contains=folder_names,
                do_fit=True,
                x_only=True,
                tomo_basis=t,
                fit_n=1.0,
                fixed=[0,2,5,6,4],
                ret_data_fit=True,
                T2star_correction=(not "Z" in t),
                LDE_element_length=7e-6
            )
            results[combi_key][dictkey][t] = data
                
            
            
# import copy
# msmts_and_data = copy.copy(msmts)

# for m in msmts.iterkeys():
#     print("Msmt: "+m)
#     ots = msmts[m]["ots"]
#     msmts_and_data[m]["tomos_data"] = dict()
#     for t in msmts[m]["tomos"]:
#         print("Tomo: %s - %s" % (m, t))
#         data = dict()
#         data["x"], data["y"], data["y_u"], data["fr"] = \
#         pu_delayfb.number_of_repetitions_stitched(
#             contains="phase_fb_delayline",
#             older_thans=ots,
#             do_fit=True,
#             x_only=True,
#             tomo_basis=t,
#             fit_n=1.0,
#             fixed=[0,2,5,6,4],
#             ret_data_fit=True
#         )
#         msmts_and_data[m]["tomos_data"][t] = data

In [ ]:
plt.figure()

from analysis.lib.fitting import fit, common

xvals = np.linspace(0,1000,1001)
for combi_key in results.iterkeys():
    for m_key in results[combi_key].iterkeys():
        if 'MBE' not in m_key:
            continue
        
        m_data = results[combi_key][m_key]
        tomos = ['XX', 'YY', 'YX', 'XY']
        bloch_length = np.zeros_like(m_data[tomos[0]]['y'])
        bloch_length_u = np.zeros_like(m_data[tomos[0]]['y'])
        for t in tomos:
            bloch_length += m_data[t]['y'] ** 2
            bloch_length_u += (m_data[t]['y_u'] * m_data[t]['y']) ** 2
        bloch_length = np.sqrt(bloch_length) / np.sqrt(2)
        bloch_length_u = np.sqrt(bloch_length_u) / bloch_length / np.sqrt(2)
        
        g_a = 0
        g_A = bloch_length[0]
        g_x0 = 0
        g_T = 500
        g_n = 1
        g_f = 0.0000
        g_phi = 0
        fixed = [0,2,4,5,6]
        
        p0, fitfunc, fitfunc_str = common.fit_exp_cos(g_a, g_A, g_x0, g_T, g_n, g_f, g_phi)

        fit_result = fit.fit1d(m_data[tomos[0]]['x'], bloch_length, None, p0=p0, fitfunc=fitfunc, do_print=True, fixed=fixed, ret=True)
        
        results[combi_key][m_key]['bloch'] = dict({
            'x': m_data[tomos[0]]['x'],
            'y': bloch_length,
            'y_u': bloch_length_u,
            'fr': fit_result
        })

        
        plt.errorbar(m_data[tomos[0]]['x'], bloch_length, bloch_length_u, label=combi_key + " " + m_key, fmt='o')
        plt.plot(xvals, fit_result['fitfunc'](xvals))
        
        
        
plt.legend()

In [ ]:
interesting_msmts = [
    ("MBE_-X", "XX", r"MBE -X, $\langle XX \rangle$"), 
    ("MBE_-X", "ZZ", r"MBE -X, $\langle ZZ \rangle$"), 
    ("MBE_-X", "bloch", r"MBE -X, equatorial bloch vector length"),
    ("MBE_X", "XX", r"MBE +X, $\langle XX \rangle$"), 
    ("MBE_X", "ZZ", r"MBE +X, $\langle ZZ \rangle$"), 
    ("MBE_X", "bloch", r"MBE +X, equatorial bloch vector length"),
    ("serial_swap_ZZ", "45.045.0", r"sep. XX, $\langle X+45\degree, X+45\degree \rangle$"),
    ("serial_swap_ZZ", "45.0-45.0", r"sep. XX, $\langle X+45\degree, X-45\degree \rangle$")
]

fit_xvals = np.linspace(0.,1000., 2000.)

colors = ['b','r','g','m','y','c','k', '0.5']

for combi in carbon_combis:
    combi_key = "".join([str(c) for c in combi])
    fig = plt.figure()
    ax = plt.subplot()
    
    i = 0
    for i_m in interesting_msmts:
        data = results[combi_key][i_m[0]][i_m[1]]
        plt.errorbar(
            data["x"], data["y"], data["y_u"], 
            fmt='o', color=colors[i], #"C%d" % i, 
            label = r"%s, $N_{1/e} = %d \pm %d$" % (i_m[2], np.round(data["fr"]['params_dict']['T']),
                                                                     np.round(data["fr"]['error_dict']['T']))
        )
        plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color=colors[i]) #"C%d" % i)
        i += 1

    plt.ylim(-1,1)
    plt.xlim(0,1000)

    plt.xlabel("Number of LDE attempts")
    plt.ylabel(r"Expectation value")
    plt.title("Correlations for carbon combination C%s" % ", C".join(combi_key))

    lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))    
    plt.savefig("batch-plots/carbon-correlations-T2star-b2-%s.png" % combi_key, dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
plt.close('all')

In [ ]:
fig = plt.figure()
ax = plt.subplot()
i=0
for c in carbons:
    c_str = str(c)
    data = single_C_decay_data[c_str]
    plt.errorbar(
        data["x"], data["y"], data["y_u"], 
        fmt='o', color="C%d" % i, 
        label = r"C%s, $N_{1/e} = %d \pm %d$" % (c_str, np.round(data["fr"]['params_dict']['T']),
                                                                 np.round(data["fr"]['error_dict']['T']))
    )
    plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color="C%d" % i)
    i += 1

plt.ylim(0,1)
plt.xlim(0,600)

plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Bloch vector length")
# plt.title("Single-nuclear state decay")

lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))    
plt.savefig("batch-plots/single-nuclear-state-decay.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()
plt.close('all')

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)

trans = "m1"

LDE_length = 7e-6
dec_time = 2.256e-6 * 2
ms0_time = LDE_length - dec_time

couplings = []
deviations = []

for c in carbons:
    print ("Carbon: %d" % c)
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    ms1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, trans)]
    avg_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_%s' % (c, trans)]
    calibrated_phase_deg = msmt_params.cfg['samples']['111no2']['C%d_phase_per_LDE_sequence_%s' % (c, trans)]
    
    parallel_coupling = ms1_freq - ms0_freq
    print("parallel coupling (kHz): \t%.3f" % (parallel_coupling*1e-3))
    
    phase = ms0_time * ms0_freq + dec_time * avg_freq
    print("phase (rotations): \t\t%.3f" % phase)
    phase_deg = (phase % 1.0) * 360.0
    print("phase (degrees): \t\t%.3f" % phase_deg)
    print("phase (calibrated): \t\t%.3f" % calibrated_phase_deg)
    
    phase_dev = phase_deg - calibrated_phase_deg
    print("phase deviation: \t\t%.3f" % phase_dev)
    
    couplings.append(parallel_coupling)
    deviations.append(phase_dev)
    
couplings = np.array(couplings) * 1e-3
deviations = np.array(deviations)

plt.scatter(couplings, deviations)

phasedevfit = np.polyfit(couplings, deviations, 1)
p = np.poly1d(phasedevfit)

plt.plot(couplings, p(couplings))
plt.xlabel("parallel hyperfine coupling (kHz)")
plt.ylabel("phase deviation (degrees)")

print("fit parameters: " + str(phasedevfit))

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)
import itertools

carbon_combis = list(itertools.combinations(carbons, 2))

freqs = np.zeros((len(carbons) + 2*len(carbon_combis), 3)) # -1, 0, +1
# T2stars = np.zeros((len(carbons) + 2*len(carbon_combis)))

c_idxs = dict()

for i_c, c in enumerate(carbons):
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    msp1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "p1")]
    msm1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "m1")]
#     T2star = msmt_params.cfg['samples']['111no2']['C%d_T2star_0' % (c, "m1")]
    
    freqs[i_c,0] = msm1_freq
    freqs[i_c,1] = ms0_freq
    freqs[i_c,2] = msp1_freq
    
#     print("C%d dephasing rate parameter: %.3f kHz" % (c, 1e-3 * (msp1_freq-msm1_freq)) )
#     T2stars[i_c] = T2star
    
    c_idxs[str(c)] = i_c
    
# 
# +1 frequencies of C4 and C5 are interchanged
# freqs[3:5,2] = freqs[3:5,2][::-1]
    
for i_cc, cc in enumerate(carbon_combis):
    idx = 2*i_cc + len(carbons)
    freqs[idx,:] = (freqs[c_idxs[str(cc[0])],:] + freqs[c_idxs[str(cc[1])],:])
#     print("C%s dephasing rate parameter: %.3f kHz" % ("%d%d+" % cc, 1e-3 * (freqs[idx,2] - freqs[idx,0])) )
    c_idxs["%d%d+" % cc] = idx
    freqs[idx+1,:] = (freqs[c_idxs[str(cc[0])],:] - freqs[c_idxs[str(cc[1])],:])
#     print("C%s dephasing rate parameter: %.3f kHz" % ("%d%d-" % cc, 1e-3 * (freqs[idx+1,2] - freqs[idx+1,0])) )
    c_idxs["%d%d-" % cc] = idx+1
    
# print freqs

In [ ]:
coupling_pairs = list(itertools.combinations(couplings, 2))

synthetic_coupling_pX = np.array([it[0] + it[1] for it in coupling_pairs])
synthetic_coupling_mX = np.array([it[0] - it[1] for it in coupling_pairs])

pX_decays = np.zeros((len(carbon_combis)))
pX_decays_u = np.zeros((len(carbon_combis)))
pX_bl_decays = np.zeros((len(carbon_combis)))
pX_bl_decays_u = np.zeros((len(carbon_combis)))
freqdiffs_pX = np.zeros((len(carbon_combis)))
mX_decays = np.zeros((len(carbon_combis)))
mX_decays_u = np.zeros((len(carbon_combis)))
mX_bl_decays = np.zeros((len(carbon_combis)))
mX_bl_decays_u = np.zeros((len(carbon_combis)))
freqdiffs_mX = np.zeros((len(carbon_combis)))

sC_decays = np.zeros((len(carbons)))
sC_decays_u = np.zeros((len(carbons)))
sC_bl_decays = np.zeros((len(carbons)))
sC_bl_decays_u = np.zeros((len(carbons)))
freqdiffs_sC = np.zeros((len(carbons)))

for i, combi in enumerate(carbon_combis):
    combi_key = "".join([str(c) for c in combi])
    pX_decays[i] = results[combi_key]['MBE_X']['XX']['fr']['params_dict']['T']
    pX_decays_u[i] = results[combi_key]['MBE_X']['XX']['fr']['error_dict']['T']
    mX_decays[i] = results[combi_key]['MBE_-X']['XX']['fr']['params_dict']['T']
    mX_decays_u[i] = results[combi_key]['MBE_-X']['XX']['fr']['error_dict']['T']
    
    pX_bl_decays[i] = results[combi_key]['MBE_X']['bloch']['fr']['params_dict']['T']
    pX_bl_decays_u[i] = results[combi_key]['MBE_X']['bloch']['fr']['error_dict']['T']
    mX_bl_decays[i] = results[combi_key]['MBE_-X']['bloch']['fr']['params_dict']['T']
    mX_bl_decays_u[i] = results[combi_key]['MBE_-X']['bloch']['fr']['error_dict']['T']
    
    freqdiffs_pX[i] = freqs[c_idxs[combi_key+"+"], 2] - freqs[c_idxs[combi_key+"+"], 0]
    freqdiffs_mX[i] = freqs[c_idxs[combi_key+"-"], 2] - freqs[c_idxs[combi_key+"-"], 0]
    
for i, c in enumerate(carbons):
    sC_decays[i] = fb_results[str(c)]['X']['fr']['params_dict']['T']
    sC_decays_u[i] = fb_results[str(c)]['X']['fr']['error_dict']['T']
    
    sC_bl_decays[i] = single_C_decay_data[str(c)]['fr']['params_dict']['T']
    sC_bl_decays_u[i] = single_C_decay_data[str(c)]['fr']['error_dict']['T']
    
    freqdiffs_sC[i] = freqs[c_idxs[str(c)], 2] - freqs[c_idxs[str(c)], 0]
    
pX_valids = pX_decays_u < 1000.0
mX_valids = mX_decays_u < 1000.0

synthetic_coupling_pX = synthetic_coupling_pX[pX_valids]
pX_decays = pX_decays[pX_valids]
pX_decays_u = pX_decays_u[pX_valids]
pX_bl_decays = pX_bl_decays[pX_valids]
pX_bl_decays_u = pX_bl_decays_u[pX_valids]
freqdiffs_pX = freqdiffs_pX[pX_valids]

synthetic_coupling_mX = synthetic_coupling_mX[mX_valids]
mX_decays = mX_decays[mX_valids]
mX_decays_u = mX_decays_u[mX_valids]
mX_bl_decays = mX_bl_decays[mX_valids]
mX_bl_decays_u = mX_bl_decays_u[mX_valids]
freqdiffs_mX = freqdiffs_mX[mX_valids]

freqdiffs_pX /= 1e3
freqdiffs_mX /= 1e3
freqdiffs_sC /= 1e3

In [ ]:
reload(common)
fig = plt.figure(figsize=(4.4,4.2))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[3, 1])
ax1 = plt.subplot(gs[0])

plt.errorbar(np.abs(synthetic_coupling_pX), pX_decays, pX_decays_u, fmt='o', label="even parity subspaces", alpha=0.5)
plt.errorbar(np.abs(synthetic_coupling_mX), mX_decays, mX_decays_u, fmt='o', label="odd parity subspaces", alpha=0.5)
plt.errorbar(np.abs(couplings), sC_decays, sC_decays_u, fmt='o', label="single nuclear spins", alpha=0.5)

combined_couplings = np.abs(np.concatenate((synthetic_coupling_pX, synthetic_coupling_mX, couplings)))
combined_decays = np.abs(np.concatenate((pX_decays, mX_decays, sC_decays)))

print(np.min(np.concatenate((pX_decays, mX_decays, sC_decays))))

decay_fit = fit.fit1d(
    combined_couplings, combined_decays, common.fit_inverse, 
    0., 1., 
    fixed=[0],
    do_print=True
)

decay_sq_fit = fit.fit1d(
    combined_couplings, combined_decays, common.fit_inverse_squared, 
    0., 1., 
    fixed=[0],
    do_print=True
)



coupl_xvals = np.linspace(0,500,1000)

plt.plot(coupl_xvals, decay_fit['fitfunc'](coupl_xvals), label='1/x fit', zorder=100, color='k')
plt.plot(coupl_xvals, decay_sq_fit['fitfunc'](coupl_xvals), #label='1/x$^2$ fit'
         zorder=100, color='k', ls='--')

plt.xlabel(r"precession frequency difference $\Delta\omega_{01}$ (kHz)")
plt.ylabel("1/e decay $N$ (LDE attempts)")

plt.legend()
# plt.ylim(0,1200)
# plt.xlim(-10,500)

ax1.set_xscale('log')
ax1.set_yscale('log')

plt.xlim(5,500)
plt.ylim(40, 1200)

ax2 = plt.subplot(gs[1])

plt.errorbar(np.abs(synthetic_coupling_pX), (pX_decays-pX_bl_decays)/pX_bl_decays, np.sqrt(pX_decays_u**2+pX_bl_decays_u**2)/pX_bl_decays, fmt='o', label="even parity subspaces", alpha=0.5)
plt.errorbar(np.abs(synthetic_coupling_mX), (mX_decays-mX_bl_decays)/mX_bl_decays, np.sqrt(mX_decays_u**2+mX_bl_decays_u**2)/mX_bl_decays, fmt='o', label="odd parity subspaces", alpha=0.5)
plt.errorbar(np.abs(couplings), (sC_decays-sC_bl_decays)/sC_bl_decays, np.sqrt(sC_decays_u**2+sC_bl_decays_u**2)/sC_bl_decays, fmt='o', label="single nuclear spins", alpha=0.5)

ax2.set_xscale('log')
plt.xlim(5,500)
plt.xlabel(r"precession frequency difference $|\Delta\omega_{01}|$ (kHz)")
plt.ylabel("relative difference \n $N_{\\langle XX \\rangle}$ and $N_{Bloch}$")
savefig("thesis-plots/decay-vs-coupling-w-residuals-loglog2")

In [ ]:
decay_fit['fitfunc'](400)

In [ ]:
diffaridoo = [(pX_decays-pX_bl_decays)/pX_bl_decays, (mX_decays-mX_bl_decays)/mX_bl_decays, (sC_decays-sC_bl_decays)/sC_bl_decays]

diffaridoo = np.concatenate(diffaridoo)

print(np.mean(diffaridoo))
print(np.std(diffaridoo))
print(len(diffaridoo))
print(np.std(diffaridoo) / np.sqrt(47))

print(np.argmin(diffaridoo))

print((list(carbons) + carbon_combis + carbon_combis)[34])

In [ ]:
reload(common)
plt.figure(figsize=(4,3))

plt.errorbar(np.abs(freqdiffs_pX), pX_decays, pX_decays_u, fmt='o', label="MBE +X")
plt.errorbar(np.abs(freqdiffs_mX), mX_decays, mX_decays_u, fmt='o', label="MBE -X")
plt.errorbar(np.abs(freqdiffs_sC), sC_decays, sC_decays_u, fmt='o', label="single carbons")

combined_couplings = np.abs(np.concatenate((freqdiffs_pX, freqdiffs_mX, freqdiffs_sC)))
combined_decays = np.abs(np.concatenate((pX_decays, mX_decays, sC_decays)))

decay_fit = fit.fit1d(
    combined_couplings, combined_decays, common.fit_inverse_squared, 
    0., 1., 
    fixed=[0],
    do_print=True
)

coupl_xvals = np.linspace(0,1000,1000)

plt.plot(coupl_xvals, decay_fit['fitfunc'](coupl_xvals), label='1/x$^2$ fit')

plt.xlabel(r"precession frequency difference $\Delta\omega_\pm$ (kHz)")
plt.ylabel("1/e decay constant (LDE attempts)")

plt.legend()
plt.ylim(0,1200)
plt.xlim(-10,300)

savefig("batch-plots/decay-vs-pmfreqdiff-zoomed-squared")

In [ ]:
plt.figure()

plt.errorbar(np.abs(synthetic_coupling_pX), pX_decays, pX_decays_u, fmt='o', label="MBE +X")
plt.errorbar(np.abs(synthetic_coupling_mX), mX_decays, mX_decays_u, fmt='o', label="MBE -X")
plt.errorbar(np.abs(couplings), sC_decays, sC_decays_u, fmt='o', label="single carbons")

combined_couplings = np.abs(np.concatenate((synthetic_coupling_pX, synthetic_coupling_mX, couplings)))
combined_decays = np.abs(np.concatenate((pX_decays, mX_decays, sC_decays)))

decay_fit = fit.fit1d(
    combined_couplings, combined_decays, common.fit_inverse, 
    0., 1., 
    fixed=[0],
    do_print=True
)

coupl_xvals = np.linspace(0,500,1000)

plt.plot(coupl_xvals, decay_fit['fitfunc'](coupl_xvals), label='1/x fit')

plt.xlabel("(Synthetic) coupling strength (kHz)")
plt.ylabel("Nuclear state 1/e decay constant (LDE attempts)")

plt.legend()
plt.ylim(0,1000)
#plt.xlim(-10,200)

plt.savefig("batch-plots/decay-vs-coupling.png", dpi=300)

In [ ]:
a = 0.5
b = 8./10.
c = (1 - b)/b**2
d = a*(1-a)
tau = 300. # ns
L = 50

In [ ]:
## import time

from scipy import stats

def calculate_phase_mean_stddev(a, b, c, d, tau, L, carbon):
    n_tries = int(1e6)
    
    Z_R = stats.norm.rvs(size=n_tries)
    Z_S = stats.norm.rvs(size=n_tries)
    Z_Sp = stats.norm.rvs(size=n_tries)
    Z_Tp = stats.norm.rvs(size=n_tries)
    Z_Tm = stats.norm.rvs(size=n_tries)
    
    R = a*L + Z_R * np.sqrt(d*L)
    S = R/b + Z_S * np.sqrt(c*R)
    Sp = S/2 + Z_Sp * np.sqrt(S/4)
    Tp = Sp*tau + Z_Tp * np.sqrt(Sp*tau**2)

    Sm = S - Sp
    Tm = Sm*tau + Z_Tm * np.sqrt(Sm*tau**2)
    
    
    
    # valids = (R > 0.0) & (S > 0.0) & (Sp > 0.0) & (Tp > 0.0) & (Sm > 0.0) & (Tm > 0.0)
    validity_matrix = np.vstack((R, S, Sp, Tp, Sm, Tm))
    validity_matrix[np.isnan(validity_matrix)] = -1.
    valids = np.all(validity_matrix > 0.0, axis=0)
    # print("valid values: " + str(np.sum(valids)))
    
    R = R[valids]
    S = S[valids]
    Sp = Sp[valids]
    Tp = Tp[valids]
    Sm = Sm[valids]
    Tm = Tm[valids]
    
    
    freq_diffs = np.diff(freqs[carbon,:]) * np.array([-1.0, 1.0])

    d_phi = (freq_diffs[0] * Tm*1e-9 + freq_diffs[1] * Tp*1e-9) * 360.0
    
    proj = np.cos(np.deg2rad(d_phi))
    
    
    
    return np.mean(d_phi), np.std(d_phi), np.mean(proj), np.std(proj)

In [ ]:
# carbon_str = "1"
# carbon = c_idxs[carbon_str]

import sys

for carbon_str, carbon in c_idxs.iteritems():
    print "Working on carbon " + carbon_str
    L = np.array([50,60,70,80,90,100,120,140,160,180,200,250,300,350,400,450,500,550,600])
    phi_means = np.zeros_like(L, dtype=np.double)
    phi_stds = np.zeros_like(L, dtype=np.double)

    proj_means = np.zeros_like(L, dtype=np.double)
    proj_std = np.zeros_like(L, dtype=np.double)

    for i_l, l in enumerate(L):
        #print("Iteration %d/%d" % (i_l+1, len(L)))
        phi_means[i_l], phi_stds[i_l], proj_means[i_l], proj_std[i_l] = calculate_phase_mean_stddev(a, b, c, d, tau, l, carbon)

    plt.figure()
    plt.scatter(L, phi_means, label='mean')
    plt.scatter(L, phi_stds, label='std dev')
    plt.xlim(0,610)
    plt.ylim(0,120)
    plt.xlabel("number of LDE attempts")
    plt.ylabel("angle (degrees)")
    plt.title("repump phase statistics on C%s" % carbon_str)
    plt.legend()
    plt.savefig("batch-plots/repump-phase-stats-T2star-C%s.png" % carbon_str, dpi=300)

    plt.figure()
    plt.scatter(L, np.deg2rad(phi_stds)**2)
    fr = fit.fit1d(L, np.deg2rad(phi_stds)**2, common.fit_line, 0, 0.01, do_print=True, fixed=[0])
    decay_N = 2. / fr['params_dict']['b']
    plt.plot(np.linspace(0,600), fr['fitfunc'](np.linspace(0,600)), label='N = %.0f' % decay_N)
    plt.legend()
    plt.xlabel("number of LDE attempts")
    plt.ylabel(r"variance in phase (degree$^2$)")
    plt.title(r"phase variance for C%s" % carbon_str)
    
    plt.savefig("batch-plots/repump-sigma-T2star-C%s.png" % carbon_str, dpi=300)

    decay_fun = lambda mu, sigma: np.cos(mu)*np.exp(-0.5*sigma**2)

    plt.figure()

    # plt.figure(figsize=(10,10))
    # plt.scatter(L, proj_means, label='sim. mean')
    # plt.scatter(L, proj_std, label='sim. std dev')
    # plt.scatter(L, decay_fun(np.deg2rad(phi_means), np.deg2rad(phi_stds)), label='mu + sigma decay')
    plt.scatter(L, decay_fun(np.deg2rad(0.0), np.deg2rad(phi_stds)), label=r'sigma decay, N = %.f' % decay_N)
    # plt.scatter(L, decay_fun(np.deg2rad(phi_means), np.deg2rad(0.0)), label='mu decay')

    if carbon < len(carbons):
        c_data = single_C_decay_data[carbon_str]
    else:
        mbe_sweep = 'MBE_X' if carbon_str[-1] == '+' else 'MBE_-X'
        c_data = results[carbon_str[:2]][mbe_sweep]['bloch']
    LDE_duration = 7e-6
    plt.errorbar(
        c_data["x"], c_data["y"], c_data["y_u"], fmt='o', 
        label='T2* corr. meas. data, N = %.f' % c_data['fr']['params_dict']['T'],
        color='r'
    )

    plt.legend()
    plt.xlim(0,610)
    plt.ylim(0,1.1)
    plt.xlabel("number of LDE attempts")
    plt.ylabel(r"$\langle X \rangle$")
    plt.title("nuclear dephasing for C%s, index %s" % (carbon_str, carbon))
    plt.savefig("batch-plots/repump-decay-T2star-C%s.png" % carbon_str, dpi=300)
    plt.close('all')
    sys.stdout.flush()

In [ ]:
interesting_msmts = [
    ("MBE_-X", "ZZ", "MBE +X, $\\langle ZZ \\rangle$"),
    ("MBE_-X", "bloch", "MBE +X, Bloch vector"),
    ("MBE_-X", "XX", "MBE +X, $\\langle XX \\rangle$"),  
    ("serial_swap_ZZ", "45.045.0", "sep. XX, $\\langle X+45\\degree, X+45\\degree \\rangle$"),
    ("MBE_X", "ZZ", "MBE -X, $\\langle ZZ \\rangle$"),
    ("MBE_X", "bloch", "MBE -X, Bloch vector"),
    ("MBE_X", "XX", "MBE -X, $\\langle XX \\rangle$"), 
    ("serial_swap_ZZ", "45.0-45.0", "sep. XX, $\\langle X+45\\degree, X-45\\degree \\rangle$")
]

fit_xvals = np.linspace(0.,1000., 2000.)

colors = ['b','r','g','m','y','c','k', '0.5']

for combi in [(2,4)]:
    combi_key = "".join([str(c) for c in combi])
    fig = plt.figure(figsize=(4,3))
    ax = plt.subplot()
    
    i = 0
    for i_m in interesting_msmts:
        data = results[combi_key][i_m[0]][i_m[1]]
        plt.errorbar(
            data["x"], data["y"], data["y_u"], 
            fmt='o', color=colors[i], #"C%d" % i, 
            label=r"%s" % (i_m[2]),
            alpha=0.5
#             label = r"%s$N_{1/e} = %d \pm %d$" % (i_m[2], np.round(data["fr"]['params_dict']['T']),
#                                                                      np.round(data["fr"]['error_dict']['T']))
        )
        plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color=colors[i], alpha=0.5) #"C%d" % i)
        i += 1

    plt.ylim(-1,1)
    plt.xlim(0,1000)

    plt.xlabel("number of LDE attempts")
    plt.ylabel(r"expectation value")
    plt.yticks([-1, -0.5, 0, 0.5, 1])
#     plt.title("Correlations for carbon combination C%s" % ", C".join(combi_key))

    lgd = plt.legend(loc=2, ncol=2, bbox_to_anchor=(-0.33,-0.2))    
#     plt.savefig("batch-plots/carbon-correlations-T2star-b2-%s.png" % combi_key, dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    savefig("thesis-plots/carbon-correlations-T2star-%s" % combi_key)
    plt.show()
plt.close('all')

In [ ]:
combi_key = "24"

for i_m in interesting_msmts:
    data = results[combi_key][i_m[0]][i_m[1]]
    print(r"%s$N_{1/e} = %d \pm %d$" % (
        i_m[2], np.round(data["fr"]['params_dict']['T']),
        np.round(data["fr"]['error_dict']['T'])
    ))
    
diff_ims = [interesting_msmts[1]] + [interesting_msmts[2]]    

i_m = diff_ims[0]
data_2 = results[combi_key][i_m[0]][i_m[1]]
i_m = diff_ims[1]
data_1 = results[combi_key][i_m[0]][i_m[1]]

print np.var(data_1["y"] - data_2["y"])


In [ ]:
(0.0210526297453+0.000905265534729) / np.sqrt(32-1)

In [ ]:
len(data_2["x"])

# PRX data and combined plotting

In [ ]:
prxE_x = np.array([ 11.60454,  13.3268 ,  15.43321,  18.43036,  18.72862,
        21.87081,  23.68491,  24.97394,  30.08122,  33.47535,  33.71853,
        37.0391 ,  38.82581,  42.26254,  48.63755,  52.73723,  55.58934,
        60.69662,  63.79975,  67.19388,  72.30116,  85.67056, 2.85211])
prxE_y = np.array([ 1006.44257556,   396.40847124,   544.62426887,
         530.59818555,   227.96299237,   204.68617807,   894.87934883,
         219.38051629,   150.08532268,   142.71120029,   248.51895129,
         209.11717177,   226.49463869,   235.05456148,   188.35843767,
          99.6751841 ,    81.21702969,    87.1328585 ,    73.66456458,
          75.31208278,    79.14954232,   104.45577687, 2213.09])
prxE_y_u = np.array([ 171.71825884,   51.70866943,   51.59627943,
         66.61537492,   25.79428152,   23.8006215 ,  153.08044075,
         34.05255351,   25.11294404,   19.23748281,   42.26161296,
         20.86384185,   30.40668479,   25.75352813,   16.66608815,
         18.90380079,   12.1637036 ,   13.20151615,   16.09665438,
         18.59659612,   11.14300614,   15.31790366, 272.932])

prxA_x = np.array([  2.84621,   8.54689,  11.63041,  13.31554,  15.43527,  18.43263,
        18.69137,  21.85612,  23.68128,  24.99615,  30.09904,  33.48653,
        33.71675,  37.02792,  38.81567,  42.26364,  48.607  ,  52.71888,
        55.58025,  60.68314,  63.82317,  67.21066,  72.31355,  85.67929])
prxA_y = np.array([  878.22416413,   801.16097336,   612.71009218,  1025.62182547,
         575.00804608,   393.83753166,   326.12566479,   227.56401883,
         419.68504625,   247.74143266,   154.81562267,   160.3387007 ,
         217.83144451,   204.18622179,   186.29345888,   196.5302694 ,
         120.18549961,   103.97877801,    96.3436406 ,    97.71962477,
          81.35130118,    72.99623642,    64.5590137 ,    69.45599937])
prxA_y_u = np.array([ 248.88175389,  119.9268169 ,  151.23202093,  186.62314688,
         36.22676371,   38.33258694,   24.02719344,   16.45082368,
         20.53652731,   35.05072338,   19.65424133,   24.83411149,
         27.82609519,   21.90512751,   19.9370088 ,   12.62164187,
         13.82636756,    9.26940316,    7.75358698,   10.73472522,
          8.98691329,   10.77613129,    4.65656129,    6.49554735])

In [ ]:
combined_couplings = np.abs(np.concatenate((synthetic_coupling_pX, synthetic_coupling_mX, couplings)))
combined_decays = np.abs(np.concatenate((pX_decays, mX_decays, sC_decays)))
combined_decays_u = np.abs(np.concatenate((pX_decays_u, mX_decays_u, sC_decays_u)))

In [ ]:
plt.figure(figsize=(4,3))

xvals = np.linspace(2.,500.,1000)

plt.errorbar(combined_couplings, combined_decays, combined_decays_u, fmt='o', label="this work, $E_{1,2}$", alpha=0.5)
plt.errorbar(prxE_x, prxE_y, prxE_y_u, fmt='o', label="PRX, $E_{1,2}$", alpha=0.5)
plt.errorbar(prxA_x, prxA_y, prxA_y_u, fmt='o', label="PRX, $A_{1,2}$", alpha=0.5)

prx_combined_x = np.concatenate((combined_couplings, prxE_x, prxA_x))
prx_combined_y = np.concatenate((combined_decays, prxE_y, prxA_y))
prx_combined_y_u = np.concatenate((combined_decays_u, prxE_y_u, prxA_y_u))

decay_fit = fit.fit1d(
    prx_combined_x, prx_combined_y, common.fit_inverse, 
    0., 1., 
    fixed=[0],
    do_print=True
)

decay_sq_fit = fit.fit1d(
    prx_combined_x, prx_combined_y, common.fit_inverse_squared, 
    0., 1., 
    fixed=[0],
    do_print=True
)

plt.plot(xvals, decay_fit['fitfunc'](xvals), ls='-', color='k', label='1/x fit')
plt.plot(xvals, decay_sq_fit['fitfunc'](xvals), ls='--', color='k', label='1/x$^2$ fit')

plt.xscale('log')
plt.yscale('log')
plt.legend(loc=1)#, bbox_to_anchor=(1.1,1.1))

plt.ylim(40,2500)

plt.xlabel("precession frequency difference $|\Delta\omega_{01}|$ (kHz)")
plt.ylabel("1/e decay N (LDE attempts)")

savefig("thesis-plots/prx-new-data-decays")

In [ ]:
np.min(prx_combined_y)